In [28]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [29]:
!pip install python-docx
!nvidia-smi
from tensorflow.keras.preprocessing.text import Tokenizer
import docx
import re

Tue May 18 07:20:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P8    35W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [36]:
class data_processing:
  
  def __init__(self):
    self.max_vocab = 1000000
    self.tokenizer = Tokenizer(num_words = self.max_vocab)
    self.text = getText("/content/drive/My Drive/sitharasongs.docx")

    self.lower_data = self.text.lower()           
    self.split_data = self.lower_data.splitlines()      
    self.final = ''     

    for line in self.split_data:
        line = clean_text(line)
        self.final += '\n' + line

    self.final_data = self.final.split('\n')       
    self.tokenizer.fit_on_texts(self.final_data)
    self.word2idx = self.tokenizer.word_index
    self.idx2word = dict(map(reversed, self.word2idx.items()))

  def return_data(self):    
      return {"final_data":self.final_data,"word2idx":self.word2idx,"idx2word":self.idx2word}
   
  def getText(self,filename):
      doc = docx.Document(filename)
      fullText = []
      for para in doc.paragraphs:
          fullText.append(para.text)
      return '\n'.join(fullText)

  def clean_text(self,text):
      text = re.sub(r',', '', text)
      text = re.sub(r'\'', '',  text)
      text = re.sub(r'\"', '', text)
      text = re.sub(r'\(', '', text)
      text = re.sub(r'\)', '', text)
      text = re.sub(r'\n', '', text)
      text = re.sub(r'“', '', text)
      text = re.sub(r'”', '', text)
      text = re.sub(r'’', '', text)
      text = re.sub(r'\.', '', text)
      text = re.sub(r';', '', text)
      text = re.sub(r':', '', text)
      text = re.sub(r'\-', '', text)
      return text
  

In [38]:
df = data_processing()
df_data = df.return_data()
print(df_data["final_data"])
print(df_data["idx2word"])
print(df_data["word2idx"])

['', 'ethu mazhayilum  udaharanam sujatha', '', 'ethu mazhayilaalumoru thirinaalam', 'raavu pakaloru pole viriyana thaaram', 'novu karipukayoothi erivirakaay', 'meenaveyilodu koode vazhi palatheri', 'paadamezhuthiya jeevakathayude ediloru lipiyaayi', 'ere vijanatha choozhum irulilum', 'aashayathu kalayaathe chiri thooki', 'mukilaayi nee niravaalin kara thedi', 'ethu mazhayilaalumoru thirinaalam', 'raavu pakaloru pole viriyana thaaram', '', 'hridayamulayum murivu mozhiyaal moodi', 'samayamani pol idayil nilkaake', 'nooru kadamakalerum chumadukalaay', 'neelamizhikalil mohamashiyaniyaathe', 'uyire nee valaraanaay kanavolam uyaranaay', 'ethu mazhayilaalumoru thirinaalam', 'raavu pakaloru pole viriyana thaaram', 'paadamezhuthiya jeevakathayude ediloru lipiyaayi', 'ere vijanatha choozhum irulilum', 'aashayathu kalayaathe chiri thooki', 'mukilaayi nee niravaalin kara thedi', '', '', 'enundodi celluloid', 'enithonnum arinjatheyille', 'punnara poonkuyile', 'nanaa nanaaa naaaaaa', 'eneno neneno 